# Bubble sort

In [1]:
:set -XNoMonadFailDesugaring
:l SortersM
:l Predicates

Similarily to selection sort, also bubble sort results in redundant sequences:

In [2]:
bubbleSortM coinCmp [3,2,1] :: [[Int]]

[[3,2,1],[3,1,2],[2,3,1],[2,1,3],[3,1,2],[3,2,1],[1,3,2],[1,2,3]]

But in this case, providing consistent predicate is still not enough:

In [3]:
import Control.Monad.State
evalStateT (bubbleSortM consistentCoinCmp [3,2,1]) noChoices :: [[Int]]

[[3,2,1],[2,3,1],[2,1,3],[3,1,2],[3,2,1],[1,3,2],[1,2,3]]

The way bubble sort algorithm works leads us to situations, when two elements are being compared in two different setting, namely `(x,y)` and `(y,x)`. What we expect from ordering relation is antisymmetry. But consistency cannot guarantee such property. Therefore, we should apply closure while recording answers in the *state*.

In [4]:
import Data.List (nub)
totalClosure :: (Eq a) => Choices a -> Choices a
totalClosure cs = nub $ cs ++ concatMap add cs
    where add ((x, y), b) = [((y, x), True) | not b]

In [5]:
totalConsistentCoinCmp :: (Eq a, MonadPlus m) => CmpMS a m
totalConsistentCoinCmp = checkChoices $ storeChoice totalClosure coinCmp

In [6]:
evalStateT (selectSortM totalConsistentCoinCmp [3,2,1]) noChoices :: [[Int]]

[[3,2,1],[2,3,1],[2,1,3],[3,1,2],[1,3,2],[1,2,3]]

We may give a try abandoning state usage, and searching for another potential place for optimizing. What is worth noticing, is what happens during bubble phase. Well, given `(x:xs)` there will be some branches, when `x` will be considered as the minimum. Then, in terms of correctness, it doesn't matter how the tail will look like. Nevertheless, in the default implementation we consider all the possible tail configurations, which leads to a great redundancy: 

In [7]:
bubbleM :: (Monad m) => CmpM a m -> [a] -> m [a]
bubbleM _ [] = return []
bubbleM _ [x] = return [x]
bubbleM p (x:xs) = do
    yss@(y:ys) <- bubbleM p xs
    b <- p x y
    return (if b then x : yss else y : x : ys)

bubbleSortM :: (Monad m) => SorterM a m
bubbleSortM _ [] = return []
bubbleSortM p xs = do
    (y:ys) <- bubbleM p xs
    fmap (y:) (bubbleSortM p ys)

evalStateT (bubbleSortM consistentCoinCmp [3,2,1]) noChoices :: [[Int]]

[[3,2,1],[2,3,1],[2,1,3],[3,1,2],[3,2,1],[1,3,2],[1,2,3]]

By simply ignoring them and providing consistency we obtain stateless, nonredundant permutation enumeration:

In [8]:
bubbleM :: (Monad m) => CmpM a m -> [a] -> m [a]
bubbleM _ [] = return []
bubbleM _ [x] = return [x]
bubbleM p (x:xs) = do
    (y:ys) <- bubbleM p xs
    b <- p x y
    return (if b then x : xs else y : x : ys)

bubbleSortM :: (Monad m) => SorterM a m
bubbleSortM _ [] = return []
bubbleSortM p xs = do
    (y:ys) <- bubbleM p xs
    fmap (y:) (bubbleSortM p ys)

evalStateT (bubbleSortM consistentCoinCmp [3,2,1]) noChoices :: [[Int]]

[[3,2,1],[2,3,1],[2,1,3],[3,1,2],[1,3,2],[1,2,3]]

However, implicitly we have just transformed bubble sort into selection sort due to giving up crucial behaviour of the bubble algorithm.